# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 28 2022 4:57PM,249638,21,RE102822,"NBTY Global, Inc.",Released
1,Oct 28 2022 4:54PM,249637,15,8184090-BO1,"Mizner Bioscience, LLC",Released
2,Oct 28 2022 4:54PM,249637,15,8381355-BO1,"Mizner Bioscience, LLC",Released
3,Oct 28 2022 4:50PM,249636,15,8184089-BO1,"Mizner Bioscience, LLC",Released
4,Oct 28 2022 4:50PM,249636,15,8126955-BO1,"Mizner Bioscience, LLC",Released
5,Oct 28 2022 4:50PM,249636,15,8184091-BO1,"Mizner Bioscience, LLC",Released
6,Oct 28 2022 4:50PM,249636,15,8279409-BO1,"Mizner Bioscience, LLC",Released
7,Oct 28 2022 4:50PM,249636,15,8279411-BO1,"Mizner Bioscience, LLC",Executing
8,Oct 28 2022 4:50PM,249636,15,8381356-BO1,"Mizner Bioscience, LLC",Executing
9,Oct 28 2022 4:50PM,249636,15,8645712-BO1,"Mizner Bioscience, LLC",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,249635,Released,6
12,249636,Executing,3
13,249636,Released,4
14,249637,Released,2
15,249638,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249634,NaN,NaN,1.0
249635,NaN,5.0,6.0
249636,NaN,3.0,4.0
249637,NaN,NaN,2.0
249638,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249587,4.0,0.0,1.0
249592,0.0,0.0,1.0
249593,1.0,1.0,1.0
249594,0.0,0.0,1.0
249597,0.0,4.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249587,4,0,1
249592,0,0,1
249593,1,1,1
249594,0,0,1
249597,0,4,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249587,4,0,1
1,249592,0,0,1
2,249593,1,1,1
3,249594,0,0,1
4,249597,0,4,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249587,4,,1
1,249592,,,1
2,249593,1,1,1
3,249594,,,1
4,249597,,4,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc."
1,Oct 28 2022 4:54PM,249637,15,"Mizner Bioscience, LLC"
3,Oct 28 2022 4:50PM,249636,15,"Mizner Bioscience, LLC"
10,Oct 28 2022 4:29PM,249635,15,"Mizner Bioscience, LLC"
21,Oct 28 2022 4:05PM,249597,15,"Mizner Bioscience, LLC"
35,Oct 28 2022 3:40PM,249634,20,"HVL, LLC dba Atrium Innovations"
36,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC"
37,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc."
40,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC"
41,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",,,1
1,Oct 28 2022 4:54PM,249637,15,"Mizner Bioscience, LLC",,,2
2,Oct 28 2022 4:50PM,249636,15,"Mizner Bioscience, LLC",,3,4
3,Oct 28 2022 4:29PM,249635,15,"Mizner Bioscience, LLC",,5,6
4,Oct 28 2022 4:05PM,249597,15,"Mizner Bioscience, LLC",,4,10
5,Oct 28 2022 3:40PM,249634,20,"HVL, LLC dba Atrium Innovations",,,1
6,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",,,1
7,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc.",1,1,1
8,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC",,,1
9,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,4,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",1,,
1,Oct 28 2022 4:54PM,249637,15,"Mizner Bioscience, LLC",2,,
2,Oct 28 2022 4:50PM,249636,15,"Mizner Bioscience, LLC",4,3,
3,Oct 28 2022 4:29PM,249635,15,"Mizner Bioscience, LLC",6,5,
4,Oct 28 2022 4:05PM,249597,15,"Mizner Bioscience, LLC",10,4,
5,Oct 28 2022 3:40PM,249634,20,"HVL, LLC dba Atrium Innovations",1,,
6,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",1,,
7,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc.",1,1,1
8,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC",1,,
9,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,1,,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",1,,
1,Oct 28 2022 4:54PM,249637,15,"Mizner Bioscience, LLC",2,,
2,Oct 28 2022 4:50PM,249636,15,"Mizner Bioscience, LLC",4,3,
3,Oct 28 2022 4:29PM,249635,15,"Mizner Bioscience, LLC",6,5,
4,Oct 28 2022 4:05PM,249597,15,"Mizner Bioscience, LLC",10,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",1,NaN,NaN
1,Oct 28 2022 4:54PM,249637,15,"Mizner Bioscience, LLC",2,NaN,NaN
2,Oct 28 2022 4:50PM,249636,15,"Mizner Bioscience, LLC",4,3.0,NaN
3,Oct 28 2022 4:29PM,249635,15,"Mizner Bioscience, LLC",6,5.0,NaN
4,Oct 28 2022 4:05PM,249597,15,"Mizner Bioscience, LLC",10,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",1,0.0,0.0
1,Oct 28 2022 4:54PM,249637,15,"Mizner Bioscience, LLC",2,0.0,0.0
2,Oct 28 2022 4:50PM,249636,15,"Mizner Bioscience, LLC",4,3.0,0.0
3,Oct 28 2022 4:29PM,249635,15,"Mizner Bioscience, LLC",6,5.0,0.0
4,Oct 28 2022 4:05PM,249597,15,"Mizner Bioscience, LLC",10,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
12,249594,1,0.0,0.0
15,1497690,24,13.0,1.0
16,249587,1,0.0,4.0
20,249634,1,0.0,0.0
21,249638,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,12,249594,1,0.0,0.0
1,15,1497690,24,13.0,1.0
2,16,249587,1,0.0,4.0
3,20,249634,1,0.0,0.0
4,21,249638,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,12,1,0.0,0.0
1,15,24,13.0,1.0
2,16,1,0.0,4.0
3,20,1,0.0,0.0
4,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,12,Released,1.0
1,15,Released,24.0
2,16,Released,1.0
3,20,Released,1.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,12,15,16,20,21
Status,,,,,
Completed,0.0,1.0,4.0,0.0,0.0
Executing,0.0,13.0,0.0,0.0,0.0
Released,1.0,24.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,12,15,16,20,21
0,Completed,0.0,1.0,4.0,0.0,0.0
1,Executing,0.0,13.0,0.0,0.0,0.0
2,Released,1.0,24.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,12,15,16,20,21
0,Completed,0.0,1.0,4.0,0.0,0.0
1,Executing,0.0,13.0,0.0,0.0,0.0
2,Released,1.0,24.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()